In [1]:
# PLOTS - inputs

# import modules
import pandas as pd
import numpy as np
import shapely as shp
import geopandas as gpd
from os.path import join as opj
import matplotlib.pyplot as plt
import os
import pickle


# suppress warnings in jupyter notebook!
import warnings
warnings.simplefilter('ignore')

def Fetch(df, key_col, key, value):
    #counties['disposal'].loc[counties['COUNTY']=='San Diego'].values[0]
    return df[value].loc[df[key_col]==key].values[0]

# set data path
DATA_DIR = "/Users/anayahall/projects/compopt/data"

print("LOADING DATA")

# read in data
# rangeland polygons
rangelands = gpd.read_file(opj(DATA_DIR, "raw/CA_FMMP_G/gl_bycounty/grazingland_county.shp"))
rangelands = rangelands.to_crs(epsg=4326)
rangelands['centroid'] = rangelands['geometry'].centroid 
rl_lon, rl_lat = rangelands.centroid.x, rangelands.centroid.y


# county polygons
county_shape = gpd.read_file(opj(DATA_DIR, 
        "raw/CA_Counties/CA_Counties_TIGER2016.shp")) # OLD- raw shape
counties_popcen = pd.read_csv(opj(DATA_DIR, 
        "counties/CenPop2010_Mean_CO06.txt")) # NEW - population weighted means!
counties_popcen.rename(columns = {'LATITUDE': 'lat', 
        'LONGITUDE': 'lon', 'COUNAME': 'COUNTY'}, inplace=True)

county_shape = county_shape.to_crs(epsg=4326)
county_shape.rename(columns = {'COUNTYFP': 'COUNTY'}, inplace=True)
county_shape['county_centroid'] = county_shape['geometry'].centroid


# solid waste inventory data (CLEANED)
swis =  gpd.read_file(opj(DATA_DIR, "clean/clean_swis.shp"))


# Minimize geodataframe to dataframe with just fields of interest
swis_df = swis[['SwisNo', 'Name', 'Latitude', 'Longitude', 'cap_m3', 'AcceptedWa']]

# rename lat and lon for easier plotting
swis_df.rename(columns = {'Latitude': 'lat', 'Longitude': 'lon'}, inplace=True)

# may just want foodwaste for adding to the plot
foodwaste_facilities = swis_df[swis_df['AcceptedWa'].str.contains("Food", na=False)]



LOADING DATA


DriverError: /Users/anayahall/projects/compopt/data/raw/CA_Counties/CA_Counties_TIGER2016.shp: No such file or directory

In [119]:
# function to group by region
def add_region_variable(df, county_column):
    '''
    Function to add a region category column to existing california dataframe (or geodataframe)
    Requires a dataframe with a column with county names and what that column is called
    Returns the same data frame with a new column describing region
    
    
    Regional categories based on CalRecycle's recommendations
    (https://www.calrecycle.ca.gov/LGCentral/Summaries/Regional/Regions/)
    
    '''
    
    df['Region'] = np.NaN

    BayArea = ['BayArea', 'Alameda', 'Contra Costa', 'Marin', 'Napa', 'San Francisco',
              'San Mateo', 'Santa Clara', 'Solano', 'Sonoma']
    CentralCoast = ['CentralCoast', 'Monterey', 'San Benito', 'San Luis Obispo',
                   'Santa Barbara', 'Santa Cruz']
    CentralValleyN = ['CentralValleyNorth', 'Butte', 'Colusa', 'Glenn', 'Placer', 
                    'Sacramento', 'Shasta', 'Sutter', 'Tehama', 'Yolo', 'Yuba']
    CentralValleyS = ['CentralValleySouth', 'Fresno', 'Kern', 'Kings', 'Madera', 'Merced', 'San Joaquin',
                     'Stanislaus', 'Tulare']
    CoastalN = ['CoastalNorth', 'Del Norte', 'Humboldt', 'Lake', 'Mendocino']
    MountainN = ['MountainNorth', 'Lassen', 'Modoc', 'Nevada', 'Plumas', 'Sierra', 'Siskiyou',
                'Trinity']
    MountainS = ['MountainSouth','Alpine', 'Amador', 'Calaveras', 'El Dorado', 'Inyo', 'Mariposa', 'Mono', 
                'Tuolumne']
    SouthernCalCoastal = ['SoCalCoastal', 'Los Angeles', 'Orange', 'San Diego', 'Ventura']
    SouthernCalInland = ['SoCalInland', 'Imperial', 'Riverside', 'San Bernardino']


    regions = np.array([bayArea, CentralCoast, CentralValleyN,
                       CentralValleyS, CoastalN, MountainS,
                       MountainN, SouthernCalCoastal, SouthernCalInland])

    for r in regions:
    #     print(r, t)
    #     print(regions[r][0])
        for county in df[county_column]:
            if np.isin(county, r):
    #             print(county, ' found in region : ', r[0])
                df.loc[df[county_column] == county,
                                    'Region'] = r[0]

    return df


In [124]:
counties_popcen = add_region_variable(counties_popcen, 'COUNTY')
county_shape =  add_region_variable(county_shape, 'NAME')
swis = add_region_variable(swis, 'County')

In [129]:
# swis['County']
regional_capacity = swis.groupby('Region').sum()


In [136]:
cd ../scripts


/Users/anayahall/projects/compopt/scripts
